In [1]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os

# positive numbers, start should be lower than end
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    indexes.reverse()
    return (indexes)

In [2]:
indexes = []
for i in rebalanceIndexes(4,40):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(indexes)

[array([-200, -197, -194, -191, -188, -185, -182, -179, -176, -173, -170,
       -167, -164, -161, -158, -155, -152, -149, -146, -143, -140, -137,
       -134, -131, -128, -125, -122, -119, -116, -113, -110, -107, -104,
       -101,  -98,  -95,  -92,  -89,  -86,  -83,  -80,  -77,  -74,  -71,
        -68,  -65,  -62,  -59,  -56,  -53]), array([-200, -197, -194, -191, -188, -185, -182, -179, -176, -173, -170,
       -167, -164, -161, -158, -155, -152, -149, -146, -143, -140, -137,
       -134, -131, -128, -125, -122, -119, -116, -113, -110, -107, -104,
       -101,  -98,  -95,  -92,  -89,  -86,  -83,  -80,  -77,  -74,  -71,
        -68,  -65,  -62,  -59,  -56,  -53,  -50]), array([-200, -197, -194, -191, -188, -185, -182, -179, -176, -173, -170,
       -167, -164, -161, -158, -155, -152, -149, -146, -143, -140, -137,
       -134, -131, -128, -125, -122, -119, -116, -113, -110, -107, -104,
       -101,  -98,  -95,  -92,  -89,  -86,  -83,  -80,  -77,  -74,  -71,
        -68,  -65,  -62,  -

In [3]:
def poolBuild(trainIndexes):
    featureList = ['Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity', 'Volatility 180 D']
    forest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = trainIndexes, target= 'Rate of Return', features = featureList, \
                                               featureLength = 12, targetLength = 3, sector = "Energy", percentileTarget = 90, percentileAvoid = 10)
    return forest

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
treeList = pool.map(poolBuild, indexes)

Index: -200
Index: -200
Index: -200
Index: -200
Index: -200
Index: -200
Index: -200
0.0 percent complete with preparing data.
0.0 percent complete with preparing data.
0.0 percent complete with preparing data.
0.0 percent complete with preparing data.
0.0 percent complete with preparing data.
Index: -200
0.0 percent complete with preparing data.
0.0 percent complete with preparing data.
0.0 percent complete with preparing data.
# of multiprocess cpus: 8


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:166: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:166: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:166: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError 

Index: -197
1.8867924528301887 percent complete with preparing data.
Index: -197
1.8518518518518516 percent complete with preparing data.
Index: -197
Index: -197
Index: -197
1.8181818181818181 percent complete with preparing data.
1.7857142857142856 percent complete with preparing data.
Index: -197
Index: -197
1.7543859649122806 percent complete with preparing data.
1.9607843137254901 percent complete with preparing data.
1.9230769230769231 percent complete with preparing data.
Index: -197
2.0 percent complete with preparing data.
Index: -194
3.7735849056603774 percent complete with preparing data.
Index: -194
3.508771929824561 percent complete with preparing data.
Index: -194
3.9215686274509802 percent complete with preparing data.
Index: -194
3.571428571428571 percent complete with preparing data.
Index: -194
Index: -194
3.7037037037037033 percent complete with preparing data.
Index: -194
3.6363636363636362 percent complete with preparing data.
3.8461538461538463 percent complete wit

In [ ]:
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	print(returns/(len(portfolio)-nancount))
	return (returns/(len(portfolio)-nancount))

def makePortfolio(treeTuple):
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Industrials")
	probabilityThreshold = 0.65
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	stocks, probabilities = zip(*stockTuples)
	getReturns(stocks, treeTuple[0], 3)
    

In [ ]:
pool.map(makePortfolio, zip(rebalanceIndexes(4,40), treeList))